## Загрузка данных

Для работы с данными импортируем библиотеку Pandas

In [4]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from collections import Counter

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

Прочитаем и сохраним данные в переменную bank_data. Сохраним размер данных перед обработкой в переменную, чтобы потом можно было оценить, какую часть данных мы потеряли после обработки.

In [5]:
bank_data = pd.read_csv('/datasets/data.csv')
raw_bank_data_shape = bank_data.shape[0]

In [6]:
raw_bank_data_shape

21525

Посмотрим как выглядят первые  10 строк в данных

In [7]:
bank_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Проверим, правильно ли указаны названия столбцов

In [8]:
bank_data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Все названия на английском, пробелов нет

Посмотрим на общую информацию о данных

In [9]:
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [10]:
bank_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Исходя из описания данных, видно следующее:
* всего имеем ~21500 записей
* в некоторых строках наблюдаются дубликаты названия категорий
* есть отрицательные значения в некоторых колонках, бессмысленные с точки зрения логики
* часть данных уже преобразована к необходимому типу, но нужно будет произвести ещё преобразования
* в двух колонках есть отсутствующие значения
* данные занимают достаточно мало памяти

## Предобработка данных

### Обработка пропусков

Посмотрим на колонку days_employed — общий трудовой стаж в днях, в ней были как отрицательные, так и пропущенные значения.
Сначала разберемся с пропущенными

In [11]:
bank_data_na = bank_data[bank_data['days_employed'].isna()]
print('Пропущено значений в колонке days_employed:', len(bank_data[bank_data['days_employed'].isna()]))
print('Доля от общих данных: {:.1%}'.format(len(bank_data[bank_data['days_employed'].isna()]) / len(bank_data)))

Пропущено значений в колонке days_employed: 2174
Доля от общих данных: 10.1%


При удалении строк с пропущенными значениями мы потеряем около 10% всех данных. Это достаточно много, поэтому попробуем подробнее посмотреть на строки с пропущенными значениями и заменить пропущенные значения, чтобы не потерять данные в важных для нас колонках.

In [12]:
bank_data_na.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Видим, что у клиентов с NaN в колонке days_employed отсутствуют значения в колонке total_income. Посмотрим сколько их

In [13]:
print('Число клиентов c пропущенным значением в колонке total_income:', len(bank_data_na[bank_data_na['total_income'].isna()]))

Число клиентов c пропущенным значением в колонке total_income: 2174


* Видим, что у одинакового числа клиентов отсутствуют значения в двух колонках. 
* Вероятно, при заполнении анкеты данные в двух колонках были указаны некорректно и в результате мы получили NaN в некоторых колонках. 
* Посмотрим, есть ли зависимость числа записей с пропуском от значений в колонках с числом детей и семейным положением.

In [14]:
def print_ratio(df, df_na, column):
    '''Функция для печати соотношения значений в наборе с пропущенными строками по отношению ко всему датасету,
    по указанной колонке'''
    df_query = dict(df[column].value_counts())
    df_na_query = dict(df_na[column].value_counts())
    
    try:
        print('Значения в колонке: {} \n'.format(column))
        for key, value in df_na_query.items():
            print('{}: {:.1%}'.format(key, value / df_query[key]))
    except:
        print('Ошибка при получении соотношения!:')
        print()

In [15]:
print_ratio(bank_data, bank_data_na, 'family_status')

Значения в колонке: family_status 

женат / замужем: 10.0%
гражданский брак: 10.6%
Не женат / не замужем: 10.2%
в разводе: 9.4%
вдовец / вдова: 9.9%


In [16]:
print_ratio(bank_data, bank_data_na, 'children')

Значения в колонке: children 

0: 10.2%
1: 9.9%
2: 9.9%
3: 10.9%
20: 11.8%
4: 17.1%
-1: 6.4%
5: 11.1%



Посмотрим, какие источники дохода у клиентов с пропусками в колонке total_income.

In [17]:
bank_data_na[bank_data_na['total_income'].isna()].groupby('income_type')['income_type'].count()

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: income_type, dtype: int64

Сгруппируем данныех без пропусков в колонке total_income и посмотрим на медиану и среднее значение.

In [18]:
grouped_bank_data_notna = bank_data[bank_data['total_income'].notna()].groupby('income_type')['total_income'].agg(['mean', 'median'])
grouped_bank_data_notna 

,mean,median
income_type,,
безработный,131339.751676,131339.751676
в декрете,53829.130729,53829.130729
госслужащий,170898.309923,150447.935283
компаньон,202417.461462,172357.950966
пенсионер,137127.465690,118514.486412
предприниматель,499163.144947,499163.144947
сотрудник,161380.260488,142594.396847
студент,98201.625314,98201.625314


Сформируем словарь из медианных значений дохода и напишем функцию для замены пропусков в колонке total_income на медианное значение при группировке по income_type.

In [19]:
grouped_mean_income_dict = grouped_bank_data_notna['mean'].to_dict()
grouped_mean_income_dict

{'безработный': 131339.7516762103,
 'в декрете': 53829.13072905995,
 'госслужащий': 170898.30992266277,
 'компаньон': 202417.46146177707,
 'пенсионер': 137127.4656901656,
 'предприниматель': 499163.1449470857,
 'сотрудник': 161380.26048788536,
 'студент': 98201.62531401133}

In [20]:
def replace_total_income_na(df, mean_dict):
    for key, value in mean_dict.items():
        try:
            df.loc[(df['total_income'].isna()) & (df['income_type'] == key), 'total_income'] = value
        except:
            print('Ошибка при замене пустого значения в категории {}, не найдены необходимые строки'.format(key))

In [21]:
replace_total_income_na(bank_data, grouped_mean_income_dict)

Нет явной зависимости между числом детей, семейным положением и пропусками в колонках days_employed и total_income. Заменим NaN значения с пропусками в колонке days_employed на 0

In [22]:
bank_data['days_employed'] = bank_data['days_employed'].fillna(0)

Проверим, правильно ли выполнили замену

In [23]:
bank_data[['days_employed', 'total_income']].isna().sum()

days_employed    0
total_income     0
dtype: int64

**Вывод**

Не обнаружили явной зависимости между пропущенными значениями и значениями в колонках, по которым необходимо сделать анализ. Причиной появления пропусков в колонках могут быть некорректно указанные значения, либо отсутствие значений ввиду отсутствия работы у клиента на момент заполнения данных. Пропущенные значения были заменены на 0 в колонке days_employed, а в колонке total_income пропущенные значения были заменены на медианные значения для соответствующей категории источника дохода.

### Замена типа данных

Обратим ещё раз внимание на значения в колонках 'days_employed' и 'total_income'

In [24]:
bank_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [25]:
bank_data[['days_employed', 'total_income']].agg(['min', 'max', 'mean', 'median'])

,days_employed,total_income
min,-18388.949901,2.066726e+04
max,401755.400475,2.265604e+06
mean,56678.874622,1.673959e+05
median,-982.531720,1.519313e+05


Значения не могут быть отрицательными и нет смысла хранить значения отработанных часов и суммы дохода в формате float, так как большой роли значения после запятой не играют. Изменим значения в обоих колонках на int.

In [26]:
bank_data[['days_employed', 'total_income']] = bank_data[['days_employed', 'total_income']].astype('int64').abs()

Проверим изменения и посмотрим на следующие значения 

In [27]:
bank_data[['days_employed', 'total_income']].agg(['min', 'max', 'mean', 'median'])

,days_employed,total_income
min,0.000000,2.066700e+04
max,401755.000000,2.265604e+06
mean,60155.970128,1.673954e+05
median,1808.000000,1.519310e+05


В значениях колонки days_employed есть очень большие значения далекие от реальных, даже если учитывать что указано не число проработанных дней, а число часов. Чтобы избавиться от таких экстремальных значений, напишем функцию, которая заменит значения в колонке days_employed на медиану, если они превышают число дней жизни человека, с учетом вычета 16 лет до возможности работать и 5-дневного рабочего дня без отпусков.

In [28]:
mean_days_employed  = int(bank_data['days_employed'].median())

In [29]:
bank_data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


In [30]:
def set_correct_days_employed(row):
        age = row['dob_years']
        days_employed = row['days_employed']
        work_days_in_year = 52 * (7 - 5)
        try:
            if days_employed > ((age - 16) * work_days_in_year):
                return mean_days_employed
        except:
            print('Что-то не так с данными:', days_employed)
        return days_employed

In [31]:
bank_data['days_employed'] = bank_data.apply(set_correct_days_employed, axis = 1)

Вернем значения в колонце к целочисленному типу

In [32]:
bank_data['days_employed'].astype('int64')

0        1808
1        1808
2        1808
3        1808
4        1808
         ... 
21520    1808
21521    1808
21522    2113
21523    1808
21524    1984
Name: days_employed, Length: 21525, dtype: int64

Посмотрим на значения в колонке children

In [33]:
bank_data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Избавимся от строк, в которых указано число детей -1 и 20, поскольку эти значения маловероятны, особенно -1

In [34]:
bank_data = bank_data[(bank_data['children'] != - 1) & (bank_data['children'] != 20)]

Посмотрим на значения в колонке dob_years

In [35]:
bank_data[bank_data['dob_years'] <= 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,1808,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291,автомобиль
149,0,1808,0,среднее,1,в разводе,3,F,сотрудник,0,70176,операции с жильем
270,3,1808,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166,ремонт жилью
578,0,1808,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620,строительство собственной недвижимости
1040,0,1808,0,высшее,0,в разводе,3,F,компаньон,0,303994,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,1808,0,среднее,1,женат / замужем,0,F,сотрудник,0,161380,жилье
20462,0,1808,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193,покупка своего жилья
20577,0,1808,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788,недвижимость
21179,2,1808,0,высшее,0,женат / замужем,0,M,компаньон,0,240702,строительство жилой недвижимости


Видим, что есть записи, с возрастом равным 0. Выберем все строки датафрейма без таких записей.

In [36]:
bank_data = bank_data[bank_data['dob_years'] > 0]

Оценим, какую часть данных мы потеряли после удаления строк.

In [37]:
print('Доля потерянных данных: {:.3%}'.format(1 - bank_data.shape[0] / raw_bank_data_shape))

Доля потерянных данных: 1.036%


**Вывод**

* Мы избавились от отрицательных значений в колонке days_employed.
* Заменили тип данных на int64 в колонках days_employed и total_income, а так же убрали нереалистичные значения стажа в колонках days_employed и children, возраста в колонке dob_years.
* После удаления строк с некорректными значениями мы потеряли 1% от общего объема данных.

### Обработка дубликатов

Посмотрим на уникальные значения в колонке education

In [38]:
bank_data['education'].value_counts()

среднее                13609
высшее                  4666
СРЕДНЕЕ                  764
Среднее                  700
неоконченное высшее      663
ВЫСШЕЕ                   270
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Видим дубликаты, записанные в различном регистре. Придем все варианты к нижнему регистру и проверим результат

In [39]:
bank_data = bank_data.copy()
bank_data['education'] = bank_data['education'].str.lower()

In [40]:
bank_data['education'].value_counts()

среднее                15073
высшее                  5202
неоконченное высшее      739
начальное                282
ученая степень             6
Name: education, dtype: int64

Посмотрим  на уникальные значения в других колонках

In [41]:
bank_data['gender'].value_counts()

F      14083
M       7218
XNA        1
Name: gender, dtype: int64

Избавимся от строки со значением пола XNA. Проверим правильность удаления

In [42]:
bank_data = bank_data[bank_data['gender'] != 'XNA']

In [43]:
bank_data['gender'].value_counts()

F    14083
M     7218
Name: gender, dtype: int64

Посмотрим  на уникальные значения в колонке family_status

In [44]:
bank_data['family_status'].value_counts()

женат / замужем          12254
гражданский брак          4138
Не женат / не замужем     2783
в разводе                 1179
вдовец / вдова             947
Name: family_status, dtype: int64

Каких-либо странных значений нет

Посмотрим на уникальные значения в колонке income_type

In [45]:
bank_data['income_type'].value_counts()

сотрудник          10996
компаньон           5033
пенсионер           3819
госслужащий         1447
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [46]:
bank_data[bank_data['income_type'] == 'безработный']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,1808,31,среднее,1,женат / замужем,0,M,безработный,1,59956,покупка жилья для сдачи
14798,0,1808,45,высшее,0,гражданский брак,1,F,безработный,0,202722,ремонт жилью


Можем заменить значения "компаньон" на "предприниматель", поскольку они несут одинаковый смысл

In [47]:
bank_data = bank_data.replace('предприниматель', 'компаньон')

In [48]:
bank_data['income_type'].value_counts()

сотрудник      10996
компаньон       5035
пенсионер       3819
госслужащий     1447
безработный        2
студент            1
в декрете          1
Name: income_type, dtype: int64

Теперь поищем дубликаты строк в датафрейме с помощью метода duplicated()

In [49]:
bank_data.duplicated().sum()

71

Видим наличие дубликатов, избавимся от них с помощью drop_duplicates() и проверим результат

In [50]:
bank_data = bank_data.drop_duplicates()
bank_data.duplicated().sum()

0

Оценим, какую часть данных мы потеряли после избавления данных от дубликатов.

In [51]:
print('Доля потерянных данных: {:.2%}'.format(1 - bank_data.shape[0] / raw_bank_data_shape))

Доля потерянных данных: 1.37%


**Вывод**

* Мы избавились от некорректных значений, дубликатов названий различных категорий в колонках и полных дубликатов строк в датафрейме.
* Причиной появления дубликатов может быть повторное ручное создание набора данных об одном и том же клиенте сотрудником банка или скриптом.
* Доля потерянных данных после предыдущих преобразований и избавления от дубликатов - 1.37%.

### Лемматизация

Посмотрим на уникальные значения в колонке purpose

In [52]:
purpose_unique_values = list(bank_data['purpose'].unique())
purpose_unique_values

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

Значений достаточно можно, но мы видим что их легко можно уменьшить в несколько раз, выделив категории, например:<br>
жилье, свадьба, образование и т.д.<br>
Поскольку одни и те же слова в колонке имеют не словарную форму и отличаются друг от друга, применим лемматизацию к значениям в колонке  purpose.
Используем для этого библиотеку pymystem3.

In [53]:
mystem = Mystem()

In [54]:
lemmas = mystem.lemmatize(' '.join(bank_data['purpose']))

Посмотрим, какие леммы встречаются чаще всего в колонке purpose

In [55]:
Counter(lemmas).most_common()

[(' ', 54473),
 ('недвижимость', 6290),
 ('покупка', 5838),
 ('жилье', 4413),
 ('автомобиль', 4258),
 ('образование', 3970),
 ('с', 2886),
 ('операция', 2576),
 ('свадьба', 2299),
 ('свой', 2212),
 ('на', 2196),
 ('строительство', 1862),
 ('высокий', 1359),
 ('получение', 1304),
 ('коммерческий', 1298),
 ('для', 1283),
 ('жилой', 1216),
 ('сделка', 933),
 ('заниматься', 900),
 ('дополнительный', 895),
 ('подержать', 838),
 ('проведение', 759),
 ('сыграть', 755),
 ('сдача', 647),
 ('семья', 636),
 ('собственный', 626),
 ('со', 623),
 ('ремонт', 602),
 ('приобретение', 457),
 ('профильный', 431),
 ('подержанный', 112),
 ('\n', 1)]

Слова "недвижимость" и "жилье" отнесём к одной категории "недвижимость". Создадим множество с самыми популярными словами

In [56]:
purpose_categories = {'недвижимость', 'автомобиль', 'образование', 'жилье', 'свадьба'}

Теперь напишем функцию для замены уникальных значений в колонке purpose на соответствующие леммы

In [57]:
def replace_purpose_value(df):
    """Присваивает строке категорию цели"""
    lemma = set(mystem.lemmatize(df['purpose']))

    intersection = list(purpose_categories & lemma)
    if not intersection:
        return 'категория не определена'
    return intersection[0]

In [ ]:
bank_data['purpose'] = bank_data.apply(replace_purpose_value, axis = 1)

Посмотрим на результат замены

In [ ]:
bank_data['purpose'].value_counts()

**Вывод**

Мы применили лемматизацию к значениям в колонке purpose и получили 4 различных цели получения кредита. Для лемматизации использовали библиотеку pymystem3.

Посмотрим на группировку наших данных по колонкам family_status и children

### Категоризация данных

Посмотрим на значения дохода в колонке total_income

In [ ]:
bank_data.describe()

Разделим доход на 3 категории:
* низкий доход между 0 и 25%
* средний доход выше 25% и ниже 75%
* высокий доход выше 75%

Для этого воспользуемся функцией qcut

In [ ]:
categorized_bank_data = bank_data.copy()
categorized_bank_data['income_group'] = pd.qcut(x=bank_data['total_income'], q=[0, .25, .75, 1], labels=['низкий доход', 'средний доход', 'высокий доход'])

Посмотрим на результат

In [ ]:
categorized_bank_data['income_group'].value_counts()

Построим сводную таблицу по колонкам family_status и children

In [ ]:
family_children_pivot = categorized_bank_data.pivot_table(index=['family_status', 'children'], values =['debt'], aggfunc=['sum'])
family_children_pivot.columns = ['Кол-во должников']
family_children_pivot

Разделим данные в колонках на несколько категорий:
* Наличие детей (колонка children):
 - Есть дети
 - Нет детей
* Семейное положение (колонка family_status):
 - Есть партнер
 - Нет партнера
 
 Это позволит использовать меньшее число категорий и легче сравнивать показатели задолженности по кредиту для различных категорий

Напишем функцию для категоризации данных в колонке children

In [ ]:
def categorize_children_column(value):
    if value == 0:
        return 'нет детей'
    return 'есть дети'

In [ ]:
categorized_bank_data['children_status'] = categorized_bank_data['children'].apply(categorize_children_column)

Напишем функцию для категоризации данных в колонке family_status

In [ ]:
def categorize_family_status_column(value):
    if value == 'Не женат / не замужем':
        return 'нет партнера'
    if value == 'в разводе':
        return 'нет партнера'
    if value == 'вдовец / вдова':
        return 'нет партнера'
    return 'есть партнер'

In [ ]:
categorized_bank_data['partner_status'] = categorized_bank_data['family_status'].apply(categorize_family_status_column)

Сформируем сводную таблицу и добавим в неё колонку с соотношением числа проблем с возвратом кредита  к общему числу записей в категории

In [ ]:
categorized_bank_data_pivot = categorized_bank_data.pivot_table(index=['partner_status', 'children_status', 'income_group'], values =['debt'], aggfunc=['count', 'sum', 'mean', lambda x: 1 - x.mean()])
categorized_bank_data_pivot.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']

Посмотрим на результаты категоризации, отобразив сводную таблицу

In [ ]:
categorized_bank_data_pivot.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

**Вывод**

Мы разделили записи в колонках children_status, family_status, total_income на несколько категорий для более удобного просмотра статистики по данным  и сформировали сводную таблицу. 

## Ответ на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на этот вопрос посмотрим на сводную таблицу

In [ ]:
children_question_pivot = categorized_bank_data.pivot_table(index=['children_status'], values =['debt'], aggfunc=['count', 'sum', 'mean', lambda x: 1 - x.mean()])
children_question_pivot.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
children_question_pivot.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

**Вывод**

Разница в доле наличия трудностей с возвратом кредита у клиентов с наличием и отсутствием детей составляет 2.3%. Чаще возвращают кредит в срок клиенты, у которых нет детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на этот вопрос посмотрим на сводную таблицу

In [ ]:
family_question_pivot = categorized_bank_data.pivot_table(index=['partner_status'], values =['debt'], aggfunc=['count', 'sum', 'mean', lambda x: 1 - x.mean()])
family_question_pivot.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
family_question_pivot.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

**Вывод**

Разница в доле наличия трудностей с возвратом кредита у клиентов с наличием и отсутствием партнера составляет 0.5%. Чаще возвращают кредит в срок клиенты, у которых есть партнер.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на этот вопрос посмотрим на сводную таблицу

In [ ]:
income_question_pivot = categorized_bank_data.pivot_table(index=['income_group'], values =['debt'], aggfunc=['count', 'sum', 'mean', lambda x: 1 - x.mean()])
income_question_pivot.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
income_question_pivot.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

**Вывод**

Максимальная разница в доле наличия трудностей с возвратом кредита у клиентов с различным доходом составляет 1.7%. Чаще всего возвращают кредит в срок клиенты с высоким доходом. Реже всего - со средним.

- Как разные цели кредита влияют на его возврат в срок?

Для ответа на этот вопрос посмотрим на сводную таблицу

In [ ]:
purpose_question_pivot = categorized_bank_data.pivot_table(index=['purpose'], values =['debt'], aggfunc=['count', 'sum', 'mean', lambda x: 1 - x.mean()])
purpose_question_pivot.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
purpose_question_pivot.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

**Вывод**

Максимальная разница в доле наличия трудностей с возвратом кредита у клиентов с различным доходом составляет 2.1%. Чаще возвращают в срок кредит, который был взят для покупки недвижимости. Реже всего - на покупку автомобиля.

## Общий вывод


* Мы провели обработку полученных данных, избавились от пропусков, дубликатов, некорректных значений. 
* Выдвинули предположение о причине появления таких значений, произвели категоризацию данных и сформировали сводную таблицу. 
* Необходимо обсудить с разработчиками формат данных каждой из колонок и исправить проблемы с пропусками и некорректными значениями в различных колонках.
* При просмотре результатов стоит учитывать, что число клиентов в некоторых группах может на несколько порядков отличаться от числа клиентов в другой группе, что может сильно влиять в ту или иную сторону при вычислении доли клиентов определенной группы, у которых были проблем с возвратом кредита в срок.
* Выяснили, какие категории клиентов чаще возвращают кредит в срок и кредит на какую цель чаще возвращают в срок:
    * Кредит на недвижимость возвращают в срок чаще
    * Клиенты, у которых есть партнер возвращают кредит в срок чаще
    * Клиенты, у которых трое детей возвращают кредит в срок чаще
    * При оценке по трем параметрам, чаще возвращают в срок кредит клиенты, у которых есть партнер, нет детей и высокий доход
    * При оценке по трем параметрам, реже возвращают в срок кредит клиенты, у которых нет партнера, есть дети и средний доход.